<a href="https://colab.research.google.com/github/mingyu1021/AI-class/blob/main/7week/CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot as plt
import numpy as np
import os

DATA_DIR = '/content/drive/MyDrive/AI-class/cifar10_data'  # CIFAR-10 데이터를 저장할 경로
# 데이터 디렉토리 생성
os.makedirs(DATA_DIR, exist_ok=True)

# --- CIFAR-10 로드 (자동으로 캐싱) ---
# Keras는 기본적으로 ~/.keras/datasets에 저장하지만,
# 명시적으로 경로를 지정하려면 아래와 같이 처리
cifar10_path = os.path.join(DATA_DIR,'cifar-10-batches-py')
if os.path.exists(cifar10_path):
    print(f"✓ 기존 데이터 발견: {cifar10_path}")
    print("저장된 데이터를 로드합니다...")
else:
    print(f"데이터가 없습니다. {DATA_DIR}에 다운로드합니다...")



데이터가 없습니다. /content/drive/MyDrive/AI-class/cifar10_data에 다운로드합니다...


In [12]:
# 데이터 로드 (없으면 자동 다운로드 후 캐싱)
(X_train, y_train),(X_test, y_test) = tf.keras.datasets.cifar10.load_data()

print(f"✓ 데이터 로드 완료")
print(f"  - 학습 데이터: {X_train.shape}")
print(f"  - 테스트 데이터: {X_test.shape}\n")



✓ 데이터 로드 완료
  - 학습 데이터: (50000, 32, 32, 3)
  - 테스트 데이터: (10000, 32, 32, 3)



In [13]:
# 클래스 이름 정의
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

# 사용할 클래스 선택
selected_classes = ['cat','dog','horse']
selected_idx = [class_names.index(c)for c in selected_classes]

# --- 해당 클래스만 필터링 ---
train_mask = np.isin(y_train, selected_idx).flatten()
test_mask = np.isin(y_test, selected_idx).flatten()

In [14]:
X_train, y_train = X_train[train_mask], y_train[train_mask]
X_test, y_test = X_test[test_mask], y_test[test_mask]

print(f"선택된 클래스: {selected_classes}")
print(f"  - 학습 샘플 수: {len(X_train)}")
print(f"  - 테스트 샘플 수: {len(X_test)}\n")


선택된 클래스: ['cat', 'dog', 'horse']
  - 학습 샘플 수: 15000
  - 테스트 샘플 수: 3000



In [15]:
# 라벨을 0~2로 다시 매핑
label_map = {v: i for i, v in enumerate(selected_idx)}
y_train = np.array([label_map[int(y)]for y in y_train])
y_test = np.array([label_map[int(y)]for y in y_test])

# 정규화
X_train, X_test = X_train / 255.0, X_test / 255.0

# 원-핫 인코딩
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (15000, 32, 32, 3)
y_train shape: (15000, 3)
X_test shape: (3000, 32, 32, 3)
y_test shape: (3000, 3)


/tmp/ipython-input-2586853951.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_train = np.array([label_map[int(y)]for y in y_train])
/tmp/ipython-input-2586853951.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_test = np.array([label_map[int(y)]for y in y_test])


In [16]:
# 모델 정의
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(selected_classes), activation='softmax')
])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │        65,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 122,115 (477.01 KB)

 Trainable params: 122,115 (477.01 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
# 학습
history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_split=0.1, verbose=2)

Epoch 1/10
106/106 - 5s - 51ms/step - accuracy: 0.4807 - loss: 0.9921 - val_accuracy: 0.5467 - val_loss: 0.9150
Epoch 2/10
106/106 - 1s - 6ms/step - accuracy: 0.6188 - loss: 0.8086 - val_accuracy: 0.6467 - val_loss: 0.7850
Epoch 3/10
106/106 - 1s - 5ms/step - accuracy: 0.6523 - loss: 0.7481 - val_accuracy: 0.6727 - val_loss: 0.7419
Epoch 4/10
106/106 - 1s - 5ms/step - accuracy: 0.6851 - loss: 0.7015 - val_accuracy: 0.6493 - val_loss: 0.7614
Epoch 5/10
106/106 - 1s - 5ms/step - accuracy: 0.7010 - loss: 0.6634 - val_accuracy: 0.7040 - val_loss: 0.6775
Epoch 6/10
106/106 - 1s - 5ms/step - accuracy: 0.7261 - loss: 0.6253 - val_accuracy: 0.7153 - val_loss: 0.6499
Epoch 7/10
106/106 - 1s - 5ms/step - accuracy: 0.7362 - loss: 0.5978 - val_accuracy: 0.7267 - val_loss: 0.6311
Epoch 8/10
106/106 - 1s - 5ms/step - accuracy: 0.7520 - loss: 0.5708 - val_accuracy: 0.7347 - val_loss: 0.6284
Epoch 9/10
106/106 - 1s - 5ms/step - accuracy: 0.7631 - loss: 0.5496 - val_accuracy: 0.7333 - val_loss: 0.6115


In [19]:
# 6. 평가
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

print('\nTest accuracy:', test_acc)

94/94 - 1s - 12ms/step - accuracy: 0.7377 - loss: 0.6097

Test accuracy: 0.737666666507721
